# Introduction to oneAPI and OpenMP* Offload with C/C++

#### Sections:
- [oneAPI Software Model Overview](#oneAPI-Software-Model-Overview)
- [HPC Single Node Workflow with oneAPI](#HPC-Single-Node-Workflow-with-oneAPI)
- Code: [Simple Exercise](#Simple-Exercise)
- [Compile and Running C/C++ Programs](#Compile-and-Running-C/C++-Programs)
- [Target Directive](#Target-Directive)
- Code: [Simple Vector Increment with Target Directive](#Lab-Exercise:-Running-an-OpenMP-program-with-the-Target-Directive)

## Learning Objectives

* Explain how __oneAPI__ can solve the challenges of programming in a heterogeneous world 
* Use oneAPI solutions to enable your workflows
* Use __OpenMP Offload__ directives to execute code on the GPU
* Familiarization on the use Jupyter notebooks for training throughout the course

### Prerequisites
This course assumes general OpenMP knowledge for CPUs. If you are new to OpenMP, below are some great resources to get you started.
* [Basic Course on OpenMP](https://www.youtube.com/watch?v=nE-xN4Bf8XI&list=PLLX-Q6B8xqZ8n8bwjGdzBJ25X2utwnoEG)
* [OpenMP Specification (for version 5.0)](https://www.openmp.org/wp-content/uploads/OpenMP-API-Specification-5.0.pdf)

## oneAPI Software Model Overview
The oneAPI software model provides a comprehensive and unified portfolio of developer tools that can
be used across hardware targets, including a range of performance libraries spanning several workload
domains. The libraries include functions custom-coded for each target architecture so the same
function call delivers optimized performance across supported architectures. oneAPI initiative is based on __industry standards and open specifications__ and is interoperable with existing HPC programming models.

<img src="Assets/oneapi2.png">

## HPC Single-Node Workflow with oneAPI 
Accelerated code can be written in either a kernel (DPC++) or __directive-based style__(OpenMP). Developers can use the __Intel® DPC++ Compatibility tool__ to perform a one-time migration from __CUDA*__ to __Data Parallel C++__. Existing __Fortran__ applications can use a __directive style based on OpenMP__. Existing __C++__ applications can choose either the __Kernel style__ or the __directive based style option__ and existing __OpenCL__ applications can remain in the OpenCL language or migrate to Data Parallel C++.

__Intel® Advisor__ is recommended to  __Optimize__ the design for __vectorization and memory__ (CPU and GPU) and __Identify__ loops that are candidates for __offload__ and project the __performance on target accelerators.__

The figure below shows the recommended approach of different starting points for HPC developers:

<img src="Assets/workflow.png">

## OpenMP vs DPC++
Both OpenMP and DPC++ are open standards that can be used to accelerate algorithms on GPUs. As the workflow diagram shows, oneAPI supports both methodologies and you should be able to achieve similar optimized performance with either option. The decision between the two choices likely depends on workflow requirements and ease of porting. When migrating from existing __CUDA__ or __OpenCL__ projects, DPC++ would likely make more sense. When migrating from existing C/Fortran applications with __OpenMP__, then OpenMP offload would be the easier alternative.

## OpenMP Offload
**OpenMP Offload** constructs are a set of directives for C++ and Fortran introduced in OpenMP 4.0 and further enhanced in later versions that allows developers to offload data and execution to target accelerators such as GPUs. OpenMP offload is supported in the Intel® oneAPI HPC Toolkit with the Intel® C++ Compiler and the Intel® Fortran Compiler.

***
## Simple Exercise
This exercise introduces OpenMP offload to the developer by way of a small simple code. In addition, it introduces the developer to the Jupyter notebook environment for editing and saving code; and for running and submitting programs to the Intel® oneAPI DevCloud.

We start with a program that includes basic OpenMP constructs including *parallel* and *for*. We will then add the *target* directive to offload part of the program to the GPU device.

This simple program loops through all of the elements of data array and multiplies it by 2. 

###  Editing the simple.cpp code
The Jupyter cell below with the gray background can be edited in-place and saved.

The first line of the cell contains the command **%%writefile 'simple.cpp'** This tells the input cell to save the contents of the cell into the file name 'simple.cpp'  As you edit the cell and run it, it will save your changes into that file.
The code below shows the simple OpenMP code. Inspect the code, there are no modifications necessary:
1. Inspect the code cell below and click run ▶ to save the code to file
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/simple.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <omp.h>

#include <iostream>

constexpr int N = 16;
int main() {
  int is_cpu;
  int *data = static_cast<int *>(malloc(N * sizeof(int)));

  // Initialization
  for (int i = 0; i < N; i++) data[i] = i;

  {
    is_cpu = omp_is_initial_device();

    // Use OpenMP to Parallelize Algorithm
#pragma omp parallel for
    for (int i = 0; i < N; i++) {
      data[i] *= 2;
    }
  }

  // Print Output
  std::cout << "Running on " << (is_cpu ? "CPU" : "GPU") << "\n";
  for (int i = 0; i < N; i++) std::cout << data[i] << "\n";

  free(data);
  return 0;
}

## Compile and Running C/C++ Programs
 
#### Compiling and Running on DevCloud:
 
For this training purposes, we have written a script (__q__) to simplify launching tasks on the DevCloud. The __q__ script does the job of submiting a script to a GPU node on DevCloud for execution, waits for the job to complete and prints out the output/errors. We will be using this command to run programs on the DevCloud: `./q <script>.sh`

#### Compiling and Running on local system:

If you have installed oneAPI HPC Toolkit on your local system, you can use the commands below to compile and run a OpenMP offload program:
```shell
source /opt/intel/inteloneapi/setvars.sh

icx -fiopenmp -fopenmp-targets=spir64 simple.cpp

./simple
  
Note: our scripts is a combination of the above three steps.
```

Using the __icx__ or __icpx__ compiler with the _"-fiopenmp -fopenmp-targets=spir64"_ options enables OpenMP offload to the GPU.

### Compile the code
To compile the code above, we'll be using the _compile_c.sh_ script. This script sets up the compile environment and executes the Intel® C++ Compiler.

In [ ]:
#Optional: Examine contents of compile_c.sh
%pycat compile_c.sh

Execute the following cell to submit the compile_c.sh script using the q script.

In [ ]:
! chmod 755 compile_c.sh; ./compile_c.sh;

_If the Jupyter cells are not responsive or if they error out when you compile the samples, please restart the Kernel and compile the samples again_

### Running the code
To execute the compiled executable, we'll be using the _run.sh_ script.

In [ ]:
#Optional: Examine contents of run.sh
%pycat run.sh

Execute the following cell to submit the run.sh script using the q script.

In [ ]:
! chmod 755 q; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q run.sh; else ./run.sh; fi

## Target Directive
The `omp target` construct transfers control and data from the host to the device. The transfer of control is sequential and synchronous. In a multi-device environment, the _device_ clause can be optionally used to denote a specific device. Each device is assigned an implementation-specific integer number. Map clauses can be used to control the direction of data flow. Map clauses will be discussed in detail in the next module.

Example:
```c
...// Sequential Host Code

#pragma omp target     //Target Region Executed on the Device
{
    for (...) {
        ...;
    }
}
...// More Sequential Host Code
```

## Lab Exercise: Running an OpenMP program with the Target Directive
In the example below, add the `#pragma omp target map(from:is_cpu) map(tofrom:data[0:N])` directive where stated to offload execution to the GPU.  We use the map clauses here to transfer *data* to and from the GPU while also copy the value of *is_cpu* back to the host to see if our code actually executed on the GPU. The map clause will be discussed in detail in the next module.

In [ ]:
%%writefile lab/simple.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <omp.h>

#include <iostream>

constexpr int N = 16;
int main() {
  int is_cpu = true;
  int *data = static_cast<int *>(malloc(N * sizeof(int)));

  // Initialization
  for (int i = 0; i < N; i++) data[i] = i;

  // Add the target directive here, including the map clause.

  {
    is_cpu = omp_is_initial_device();
#pragma omp parallel for
    for (int i = 0; i < N; i++) {
      data[i] *= 2;
    }
  }

  // Print Output
  std::cout << "Running on " << (is_cpu ? "CPU" : "GPU") << "\n";
  for (int i = 0; i < N; i++) std::cout << data[i] << "\n";

  free(data);
  return 0;
}

In [ ]:
# Execute this cell to compile the code
! chmod 755 compile_c.sh; ./compile_c.sh;

In [ ]:
# Execute this cell to run the code
! chmod 755 q; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q run.sh; else ./run.sh; fi

_If the Jupyter cells are not responsive or if they error out when you compile the code samples, please restart the Jupyter Kernel: 
"Kernel->Restart Kernel and Clear All Outputs" and compile the code samples again_

Once execution completes, you should see the message that the the program ran on the GPU.

In [ ]:
# See the solution by running this cell
%pycat simple_solution.cpp

# Summary
In this module you have learned the following:
* How oneAPI solves the challenges of programming in a heterogeneous world 
* Take advantage of oneAPI solutions to enable your workflows
* Use the Intel DevCloud to test drive oneAPI tools and libraries
* Introduced to the target directive to enable OpenMP offload
* Become familiarized with the use of Juypter notebooks by editing of source code in context.

## Resources

Check out these related resources

#### Intel® oneAPI
* [oneAPI main page](https://software.intel.com/oneapi "oneAPI main page")
* [Intel® DevCloud](https://software.intel.com/en-us/devcloud/oneapi "Intel DevCloud")
* [Get Started with oneAPI for Linux*](https://software.intel.com/en-us/get-started-with-intel-oneapi-linux)
* [Get Started with oneAPI for Windows*](https://software.intel.com/en-us/get-started-with-intel-oneapi-windows)
* [oneAPI Release Notes](https://software.intel.com/en-us/articles/intel-oneapi-release-notes)
* [oneAPI Sample Codes](https://software.intel.com/en-us/articles/code-samples-for-intel-oneapibeta-toolkits)

#### OpenMP
* [OpenMP Specification (for version 5.0)](https://www.openmp.org/wp-content/uploads/OpenMP-API-Specification-5.0.pdf)

***

@Intel Corporation | [\*Trademark](https://www.intel.com/content/www/us/en/legal/trademarks.html)